In [1]:
#libraries 
import os
#The OS: functions for interacting with the operating system
import pandas as pd
from datetime import datetime as dt
from datetime import time as t
import numpy as np
import datetime
from IPython.display import display

In [2]:
# source
personal_path = os.environ['USERPROFILE']
middle_path = r'Concentrix Corporation\CNXVN - WFM Team - Tài liệu\DataBase'
user_credential = os.path.join(os.environ['USERPROFILE'], middle_path)

if 1==1: # this condition is to hide the links
    masterroster_path = os.path.join(user_credential, r'DataFrame\BKN\DataJSON\BKN\masterroster.json')
    schedule_path = os.path.join(user_credential, r'DataFrame\BKN\DataJSON\BKN\schedule.json')
    GI_path = os.path.join(user_credential, r'DataFrame\BKN\DataJSON\BKN\gi.json')
    PI_path = os.path.join(user_credential, r'DataFrame\BKN\DataJSON\BKN\pi.json')
    CSP_path = os.path.join(user_credential, r'DataFrame\BKN\DataJSON\BKN\csp.json')
    ub_accounts_path = os.path.join(user_credential, r'DataFrame\BKN\DataJSON\BKN\ub_accounts.json')    

In [3]:
# IMPORT GI PI
gi = pd.read_json(GI_path)
gi['EID'] = gi['EID'].astype("int64")
pi = pd.read_json(PI_path)
pi['EID'] = pi['EID'].astype("int64")
csp = pd.read_json(CSP_path)
csp['EID'] = csp['EID'].astype("int64")


In [4]:
# IMPORT UB Accounts
ub_accounts = pd.read_json(ub_accounts_path)
ub_accounts['Emp ID'] = ub_accounts['Emp ID'].astype("int64")
ub_accounts['API Key'] = ub_accounts['API Key']


In [5]:
# IMPORT MASTER ROSTER
masterroster = pd.read_json(masterroster_path)
masterroster['Employee_ID'] = masterroster['Employee_ID'].astype("int64")
masterroster = masterroster[['Employee_ID','Wave #','Booking Login ID','TED Name','BKN_ID','Booking Email']]
# masterroster.info()

In [6]:
# IMPORT SCHEDULE
schedule = pd.read_json(schedule_path)
schedule = schedule[['Emp ID','Date','LOB','team_leader']]
schedule['Week'] = schedule['Date'].dt.strftime('%Y%W').astype('int64')
# schedule['Year'] = schedule['Date'].dt.strftime('%Y').astype('int64')
schedule = pd.merge(schedule, masterroster, left_on='Emp ID', right_on='Employee_ID', how='left')
schedule = schedule.drop(columns=['Employee_ID','Date'])
schedule = schedule.drop_duplicates(['Emp ID','Week','team_leader'])

# Get TED name's TL
schedule = pd.merge(schedule, masterroster[['Employee_ID','TED Name']], left_on=['team_leader'], right_on='Employee_ID', how='left')
schedule = schedule.rename(columns={'TED Name_y':'TL','TED Name_x':'TED Name'})
schedule = schedule.drop(columns=['team_leader','Employee_ID'])


# Get GI PI CSP
schedule = pd.merge(schedule, gi['EID'], left_on=['Emp ID'], right_on=['EID'], how='left')
schedule = schedule.rename(columns={'EID':'GI trained'})
schedule = pd.merge(schedule, pi['EID'], left_on=['Emp ID'], right_on=['EID'], how='left')
schedule = schedule.rename(columns={'EID':'PI trained'})
schedule = pd.merge(schedule, csp['EID'], left_on=['Emp ID'], right_on=['EID'], how='left')
schedule = schedule.rename(columns={'EID':'CSP trained'})


schedule['GI trained'] = schedule['GI trained'].fillna(0).astype('int64')
schedule['PI trained'] = schedule['PI trained'].fillna(0).astype('int64')
schedule['CSP trained'] = schedule['CSP trained'].fillna(0).astype('int64')


# schedule.to_excel(os.path.join(user_credential, r'DataRaw\BKN\Change LOB\schedule.xlsx'), index=False)
# schedule.info()

In [7]:
def get_weeks_in_year(year):
    # Initialize the dictionary to store weeks and their days
    weeks = {}
    
    # Get the first day of the year
    first_day = datetime.date(year, 1, 1)
    # Find the first Monday of the year
    first_monday = first_day + datetime.timedelta(days=(7 - first_day.weekday()) % 7)
    current_monday = first_monday
    
    # Iterate through all days of the year
    for day in range(365 + (1 if datetime.date(year, 12, 31).strftime('%j') == '366' else 0)):
        current_day = current_monday + datetime.timedelta(days=day)
        week_num = current_day.strftime('%Y%W')
        
        if week_num not in weeks:
            weeks[week_num] = []
        
        weeks[week_num].append(current_day.strftime('%Y-%m-%d'))
        
    
    return weeks

# Get the dictionary of weeks and their days for the current year and next year
weeks_in_cur_year = get_weeks_in_year(2025)
weeks_in_next_year = get_weeks_in_year(2025)

# Print the result
for c_week, c_days in weeks_in_cur_year.items():
    print(f"{c_week}: {c_days}")


for n_week, n_days in weeks_in_next_year.items():
    print(f"{n_week}: {n_days}")

202501: ['2025-01-06', '2025-01-07', '2025-01-08', '2025-01-09', '2025-01-10', '2025-01-11', '2025-01-12']
202502: ['2025-01-13', '2025-01-14', '2025-01-15', '2025-01-16', '2025-01-17', '2025-01-18', '2025-01-19']
202503: ['2025-01-20', '2025-01-21', '2025-01-22', '2025-01-23', '2025-01-24', '2025-01-25', '2025-01-26']
202504: ['2025-01-27', '2025-01-28', '2025-01-29', '2025-01-30', '2025-01-31', '2025-02-01', '2025-02-02']
202505: ['2025-02-03', '2025-02-04', '2025-02-05', '2025-02-06', '2025-02-07', '2025-02-08', '2025-02-09']
202506: ['2025-02-10', '2025-02-11', '2025-02-12', '2025-02-13', '2025-02-14', '2025-02-15', '2025-02-16']
202507: ['2025-02-17', '2025-02-18', '2025-02-19', '2025-02-20', '2025-02-21', '2025-02-22', '2025-02-23']
202508: ['2025-02-24', '2025-02-25', '2025-02-26', '2025-02-27', '2025-02-28', '2025-03-01', '2025-03-02']
202509: ['2025-03-03', '2025-03-04', '2025-03-05', '2025-03-06', '2025-03-07', '2025-03-08', '2025-03-09']
202510: ['2025-03-10', '2025-03-11', 

In [8]:
# Function to get week number if date matches a value in the dictionary
# Select a date from the dict
date_choosen = datetime.date.today().strftime('%Y-%m-%d')
next_date_choosen = (datetime.date.today() + datetime.timedelta(days=4)).strftime('%Y-%m-%d')

# Compare the choosen date with each item in the dict
for key, values in weeks_in_cur_year.items():
    if date_choosen in values:
        print(f'{key}: {date_choosen}')
        current_week = int(key)



for next_key, values in weeks_in_cur_year.items():
    if next_date_choosen in values:
        print(f'{next_key}: {next_date_choosen}')
        next_week = int(next_key)
    else:
        for next_key, values in weeks_in_next_year.items():
            if next_date_choosen in values:
                next_week = int(next_key)
                print(f'{next_key}: {next_date_choosen}')


202516: 2025-04-27
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025-05-01
202517: 2025

In [9]:
# Compare LOB
sched1 = schedule[schedule['Week'] == current_week]


sched2 = schedule[schedule['Week'] == next_week]

change_lob = pd.merge(sched1[['Emp ID','LOB','Week']], sched2, left_on=['Emp ID'], right_on=['Emp ID'], how='right')

change_lob = change_lob.rename(columns={'Week_x':'Current week','Week_y':'Requested week',
                                        'LOB_x':'Current Language/Profile','LOB_y':'Requested LOB','BKN_ID':'Staff ID'})
# change_lob = change_lob.loc[(change_lob['Current week'] == week_num) & (change_lob['Current year'] == year)]

                                            

def check_diff(x):
    if x['Current Language/Profile'] == x['Requested LOB']:
        return 'True'
    else:
        return 'False'

change_lob['Check LOB'] = change_lob.apply(check_diff, axis=1)
change_lob['Current Language/Profile'] = change_lob.apply(lambda x: 'New Hired' if x['Current Language/Profile'] == '' else x['Current Language/Profile'], axis=1)


conditions = [(change_lob['Requested LOB'] == 'EN'),    (change_lob['Requested LOB'] == 'EN EX'),        (change_lob['Requested LOB'] == 'VICSG'), 
              (change_lob['Requested LOB'] == 'VICSP'), (change_lob['Requested LOB'] == 'Senior VICSP'), (change_lob['Requested LOB'] == 'NL'), 
              (change_lob['Requested LOB'] == 'HE4'),   (change_lob['Requested LOB'] == 'XT4'),          (change_lob['Requested LOB'] == 'ID4'), 
              (change_lob['Requested LOB'] == 'TR'),    (change_lob['Requested LOB'] == 'EL'),           (change_lob['Requested LOB'] == 'KO'), 
              (change_lob['Requested LOB'] == 'CS'), (change_lob['Requested LOB'] == 'UB PL'),        (change_lob['Requested LOB'] == 'UB HU'), 
              (change_lob['Requested LOB'] == 'RU'), (change_lob['Requested LOB'] == 'UB ZH'),        (change_lob['Requested LOB'] == 'FR'), 
              (change_lob['Requested LOB'] == 'PT'),    (change_lob['Requested LOB'] == 'IT'),           (change_lob['Requested LOB'] == 'NO'), 
              (change_lob['Requested LOB'] == 'DE'),    (change_lob['Requested LOB'] == 'DA'),           (change_lob['Requested LOB'] == 'RO')]
values = ['English','EN EX','Vietnamese CSG','CS Partner Specialist','Outsourced EPS Senior Partner Specialist','Unbabel - Dutch','Unbabel - Hebrew','Unbabel - Chinese','Unbabel - Indonesian',
          'Unbabel - Turkish','Unbabel - Greek','Unbabel - Korean','Unbabel - Czech','Unbabel - Polish','Unbabel - Hungarian','Unbabel - Russian',
          'Unbabel - Chinese','Unbabel - France','Unbabel - Portuguese','Unbabel - Italian','Unbabel - Norway','Unbabel - German','Unbabel - Denmark',
          'Unbabel - Romanian']    
change_lob['Requested Language'] = np.select(conditions, values)


# Define PEGA specialist or CS Specialist
def pega_full_trained(x):
    if x['Check LOB'] == 'False':
        if x['Requested Language'] == 'English':
            if x['PI trained'] != 0:
                return 'EN PEGA Guest Specialist'
            else:
                return 'EN PEGA Guest Specialist'
        else:
            return x['Requested Language']
    else:
        return
    
change_lob['Requested Language/Profile'] = change_lob.apply(pega_full_trained, axis=1)
change_lob = change_lob.loc[change_lob['Check LOB'] == 'False']




change_lob = pd.merge(change_lob, ub_accounts[['Emp ID','Username','API Key']], left_on='Emp ID', right_on='Emp ID', how='left')


# #########
# Thanhtest = change_lob
# Thanhtest.to_csv(os.path.join(user_credential, r'C:\Users\hathanh.le\Concentrix Corporation\CNXVN - WFM Team - Documents\RTA_PersonalFile\Thanh\pha\Thanhtest.csv'), index=False)
# #########


change_lob = change_lob[['TL','TED Name','Wave #','Staff ID','Current Language/Profile','Requested Language/Profile','Requested LOB',
                         'Emp ID','Booking Login ID','Booking Email','Username','API Key']]

# Get requested date
change_lob['Request date'] = datetime.date.today()

change_lob.to_excel(os.path.join(user_credential, r'DataRaw\BKN\Change LOB\change_lob.xlsx'), index=False)


In [10]:
# Append Change LOB file to Ticket record weekly
os.path.join(user_credential, r'DataRaw\BKN\Change LOB\Ticket record weekly.xlsx')

df1 = pd.read_excel(os.path.join(user_credential, r'DataRaw\BKN\Change LOB\Ticket record weekly.xlsx'))
df2 = pd.read_excel(os.path.join(user_credential, r'DataRaw\BKN\Change LOB\change_lob.xlsx'))

df_merged = pd.concat([df1, df2])
df_merged.to_excel(os.path.join(user_credential, r'DataRaw\BKN\Change LOB\Ticket record weekly.xlsx'), index=False)


In [11]:
# Get creation date
filename = os.path.join(user_credential, r'DataRaw\BKN\Change LOB\change_lob.xlsx')
c_time = os.path.getctime(filename)
c_date_time = dt.fromtimestamp(c_time).strftime('%Y%m%d_%H%M%S')
c_date_time = f"{c_date_time}.xlsx"


# Function to rename file by create date time

def rename_files_by_date(directory, condition):
    # Check if it's a file (not a directory)
    if os.path.isfile(filename):

        # Get the creation date time of the file
        c_time = os.path.getctime(filename)
        c_date_time = dt.fromtimestamp(c_time).strftime('%Y%m%d_%H%M%S')
        new_filename = f"{c_date_time}.xlsx"

        # Check if the condition is met
        if condition(new_filename):
            new_filepath = os.path.join(directory, new_filename)
            # Rename the file
            os.rename(filename, new_filepath)


def condition(new_filename):
    return new_filename


# Specify the directory containing the files to be renamed
directory = os.path.join(user_credential, r'DataRaw\BKN\Change LOB')


# Call the function to rename files
rename_files_by_date(directory, condition)